# How-To, First Steps

## Load package and create basic options

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from robustfpm.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [2]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [3]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

Option 1: <robustfpm.finance.derivatives.AmericanOption object at 0x10f13a400>
Option 2: <robustfpm.finance.derivatives.EuropeanOption object at 0x10f13a760>
Option 3: <robustfpm.finance.derivatives.BermudanOption object at 0x11dcdee80>
Option 4: <robustfpm.finance.derivatives.AmericanOption object at 0x11dcdeeb0>


In [4]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

Option 1 payoff at x = 80, 90, and 100, t = 3: [10.  0.  0.]
Option 2 payoff at x = 100, t = 3: [-inf]
Option 3 payoff at x = 100, t = 3: [10.]
Option 3 payoff at x = 100, t = 4: [-inf]


## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `robustfpm.pricing`

In [5]:
from robustfpm.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [6]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [7]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.1178 sec (CPU 0.1177 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 19/90 (21.11%)
iter = 78/90 (86.67%)
iter = 87/90 (96.67%)
t = 3
iter = 59/67 (88.06%)
t = 2
t = 1
t = 0
Computing value function in intermediate points in time: 0.4632 sec (CPU 0.4632 sec)
Solving the problem: 0.5823 sec (CPU 0.5823 sec)
Value: 5.327786420219619


Now let's also calculate and check precision

In [8]:
sol1 = solver.solve(pm1, calc_precision = True, do_precalc=False)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.0866 sec (CPU 0.0866 sec)
Computing A^* and Lipschitz constants for constraints: 0.0002 sec (CPU 0.0002 sec)
Calculating precision: 0.0002 sec (CPU 0.0002 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
t = 3
t = 2
iter = 41/45 (91.11%)
t = 1
t = 0
Computing value function in intermediate points in time: 0.4711 sec (CPU 0.4711 sec)
Solving the problem: 0.5590 sec (CPU 0.5591 sec)
Value: 5.327786420219619


In [9]:
print('Precision: {0}'.format(sol1['precision']))

Precision: [5.85182876 4.00129318 2.49774194 1.33225806 0.5        0.        ]


Let's play around and change Lattice step, especially since the error is too big.

In [10]:
pm1.lattice = Lattice(delta=[.1])
solver.reset()
sol2 = solver.solve(pm1, calc_precision = True)
print('Value: {0}'.format(sol2['Vf'][0][0]))
print('Precision: {0}'.format(sol2['precision']))

Precalculating points for value function evaluation: 0.7388 sec (CPU 0.7388 sec)
Computing A^* and Lipschitz constants for constraints: 0.0004 sec (CPU 0.0004 sec)
Calculating precision: 0.0002 sec (CPU 0.0002 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 52/818 (6.36%)
iter = 170/818 (20.78%)
iter = 196/818 (23.96%)
iter = 207/818 (25.31%)
iter = 218/818 (26.65%)
iter = 227/818 (27.75%)
iter = 330/818 (40.34%)
iter = 394/818 (48.17%)
iter = 398/818 (48.66%)
iter = 543/818 (66.38%)
iter = 575/818 (70.29%)
iter = 595/818 (72.74%)
iter = 614/818 (75.06%)
iter = 616/818 (75.31%)
iter = 650/818 (79.46%)
iter = 657/818 (80.32%)
iter = 730/818 (89.24%)
iter = 738/818 (90.22%)
iter = 752/818 (91.93%)
t = 3
iter = 64/609 (10.51%)
iter = 156/609 (25.62%)
iter = 397/609 (65.19%)
iter = 603/609 (99.01%)
t = 2
iter = 15/405 (3.70%)
iter = 266/405 (65.68%)
iter = 280/405 (69.14%)
t = 1
iter = 38/203 (18.72%)
iter = 43/203 (21.18%)
iter = 61/203 (30.05%)
i

And what if we go to the extreme?

In [11]:
pm1.lattice = Lattice(delta=[.01])
solver.reset() # otherwise, we get old precision
precision = solver.get_precision(pm1)
print('Precision: {0}'.format(precision))

Precalculating points for value function evaluation: 8.8045 sec (CPU 8.8046 sec)
Computing A^* and Lipschitz constants for constraints: 0.0016 sec (CPU 0.0016 sec)
Calculating precision: 0.0005 sec (CPU 0.0005 sec)
Precision: [0.05427884 0.03741037 0.02364124 0.01286389 0.005      0.        ]


In [12]:
solver.iter_tick = 1000
sol3 = solver.solve(pm1, do_precalc=False)
print('Value: {0}'.format(sol3['Vf'][0][0]))

Computing value function in the last point: 0.0006 sec (CPU 0.0006 sec)
t = 4
iter = 1322/8089 (16.34%)
iter = 1415/8089 (17.49%)
iter = 2529/8089 (31.26%)
iter = 8071/8089 (99.78%)
t = 3
iter = 132/6027 (2.19%)
iter = 696/6027 (11.55%)
iter = 1813/6027 (30.08%)
iter = 3267/6027 (54.21%)
iter = 4036/6027 (66.97%)
iter = 4403/6027 (73.05%)
t = 2
iter = 539/4005 (13.46%)
iter = 1062/4005 (26.52%)
iter = 1110/4005 (27.72%)
iter = 2805/4005 (70.04%)
iter = 3957/4005 (98.80%)
t = 1
iter = 46/2003 (2.30%)
iter = 1435/2003 (71.64%)
t = 0
Computing value function in intermediate points in time: 47.9600 sec (CPU 47.9607 sec)
Solving the problem: 47.9610 sec (CPU 47.9617 sec)
Value: 4.322815140670112


A significant gain in precision — but not so much in the Value...

That's because we give a *guaranteed* estimate for precision that depends on grid and assumes that the $v_t$ is the *worst*.

Let's try 2D Problem with another option and *additive* dynamics.

In [13]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 500
solver.reset()
sol5 = solver.solve(pm2, calc_precision = True)
print('Value: {0}'.format(sol5['Vf'][0][0]))
print('Precision: {0}'.format(sol5['precision']))

Precalculating points for value function evaluation: 0.8877 sec (CPU 0.8877 sec)
Computing A^* and Lipschitz constants for constraints: 0.0008 sec (CPU 0.0008 sec)
Calculating precision: 0.0001 sec (CPU 0.0001 sec)
Computing value function in the last point: 0.0004 sec (CPU 0.0004 sec)
t = 4
iter = 100/5913 (1.69%)
iter = 310/5913 (5.24%)
iter = 439/5913 (7.42%)
iter = 987/5913 (16.69%)
iter = 1222/5913 (20.67%)
iter = 2048/5913 (34.64%)
iter = 2933/5913 (49.60%)
iter = 3935/5913 (66.55%)
iter = 4538/5913 (76.75%)
iter = 4913/5913 (83.09%)
iter = 5137/5913 (86.88%)
iter = 5275/5913 (89.21%)
iter = 5331/5913 (90.16%)
iter = 5800/5913 (98.09%)
t = 3
iter = 9/3355 (0.27%)
iter = 38/3355 (1.13%)
iter = 74/3355 (2.21%)
iter = 1044/3355 (31.12%)
iter = 1516/3355 (45.19%)
iter = 1668/3355 (49.72%)
iter = 1680/3355 (50.07%)
iter = 1962/3355 (58.48%)
iter = 2712/3355 (80.83%)
iter = 2726/3355 (81.25%)
iter = 2865/3355 (85.39%)
iter = 2871/3355 (85.57%)
t = 2
iter = 659/1517 (43.44%)
iter = 893/

In [14]:
pm3 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 500
solver.reset()
sol6 = solver.solve(pm3, calc_precision = True)
print('Value: {0}'.format(sol6['Vf'][0][0]))
print('Precision: {0}'.format(sol6['precision']))

Precalculating points for value function evaluation: 0.8502 sec (CPU 0.8502 sec)
Computing A^* and Lipschitz constants for constraints: 0.0010 sec (CPU 0.0010 sec)
Calculating precision: 0.0001 sec (CPU 0.0001 sec)
Computing value function in the last point: 0.0006 sec (CPU 0.0006 sec)
t = 4
iter = 295/5913 (4.99%)
iter = 663/5913 (11.21%)
iter = 1664/5913 (28.14%)
iter = 1828/5913 (30.91%)
iter = 2342/5913 (39.61%)
iter = 2957/5913 (50.01%)
iter = 3161/5913 (53.46%)
iter = 3395/5913 (57.42%)
iter = 3402/5913 (57.53%)
iter = 4115/5913 (69.59%)
iter = 4124/5913 (69.74%)
iter = 4186/5913 (70.79%)
iter = 4206/5913 (71.13%)
iter = 4574/5913 (77.35%)
iter = 4939/5913 (83.53%)
iter = 5047/5913 (85.35%)
iter = 5107/5913 (86.37%)
iter = 5329/5913 (90.12%)
t = 3
iter = 118/3355 (3.52%)
iter = 1049/3355 (31.27%)
iter = 1076/3355 (32.07%)
iter = 1089/3355 (32.46%)
iter = 2246/3355 (66.94%)
iter = 2352/3355 (70.10%)
iter = 3082/3355 (91.86%)
iter = 3090/3355 (92.10%)
t = 2
iter = 10/1517 (0.66%)
i

In [15]:
pm4 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=LongOnlyConstraints,
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 500
solver.reset()
sol7 = solver.solve(pm4, calc_precision = True)
print('Value: {0}'.format(sol7['Vf'][0][0]))
print('Precision: {0}'.format(sol7['precision']))

Precalculating points for value function evaluation: 0.9063 sec (CPU 0.9063 sec)
Computing A^* and Lipschitz constants for constraints: 16.6683 sec (CPU 16.6641 sec)
Calculating precision: 0.0001 sec (CPU 0.0001 sec)
Computing value function in the last point: 0.0008 sec (CPU 0.0008 sec)
t = 4
iter = 949/5913 (16.05%)
iter = 1526/5913 (25.81%)
iter = 2412/5913 (40.79%)
iter = 2616/5913 (44.24%)
iter = 3361/5913 (56.84%)
iter = 4184/5913 (70.76%)
iter = 4397/5913 (74.36%)
t = 3
iter = 372/3355 (11.09%)
iter = 419/3355 (12.49%)
iter = 934/3355 (27.84%)
iter = 1111/3355 (33.11%)
iter = 1514/3355 (45.13%)
t = 2
iter = 1205/1517 (79.43%)
iter = 1314/1517 (86.62%)
t = 1
iter = 10/399 (2.51%)
t = 0
Computing value function in intermediate points in time: 46 min 24.3658 sec (CPU 46 min 24.3499 sec)
Solving the problem: 46 min 41.9433 sec (CPU 46 min 41.9232 sec)
Value: 5.000000155687845
Precision: [23.12112905  9.91527857  3.71483598  1.01709734  0.07071068  0.        ]
